In [47]:
import pandas as pd
import gmaps
import numpy as np
import re
import requests
import json
import time
import urllib
import urllib.request as urllib2


from config import gkey
gmaps.configure(api_key=gkey)

In [2]:
# ZHVI All Homes (SFR, Condo/Co-op) Time Series ($) (by ZIP code)
csv_home_values = "Resources/Zip_Zhvi_AllHomes.csv"

# Median Sale Price - Seasonally Adjusted ($) (by ZIP code)
csv_home_sales = "Resources/Sale_Prices_Zip.csv"

# Monthly Home Sales (Number, Raw) (by ZIP code)
csv_number_sales = "Resources/Sale_Counts_Zip.csv"

# Foreclosure Resales (%) (by ZIP code)
csv_foreclosure_resales = "Resources/SalesPrevForeclosed_Share_Zip.csv"

# Unemployment data
csv_unemployment = "Resources/Cleaned_BLS_Unemployment-Rate-by-State_2008-2019_1-11-2020.csv"

home_values = pd.read_csv(csv_home_values, encoding="ISO-8859-1")
home_sales = pd.read_csv(csv_home_sales, encoding="ISO-8859-1")
number_sales = pd.read_csv(csv_number_sales, encoding="ISO-8859-1")
foreclosure_resales = pd.read_csv(csv_foreclosure_resales, encoding="ISO-8859-1")
unemployment = pd.read_csv(csv_unemployment)

# Suffixes:
# ZVHI = Zillow Home Value Index ($)
# MSR = Median Sales Price - Seasonally Adjusted ($)
# NS = Number of Sales
# FR = Foreclosure Resales (%)
# UNEMP = Unemployment


In [3]:
home_values["2008-Avg_ZHVI"] = home_values.iloc[:, 7:19].mean(axis=1)
home_values["2009-Avg_ZHVI"] = home_values.iloc[:, 19:31].mean(axis=1)
home_values["2010-Avg_ZHVI"] = home_values.iloc[:, 31:43].mean(axis=1)
home_values["2011-Avg_ZHVI"] = home_values.iloc[:, 43:55].mean(axis=1)
home_values["2012-Avg_ZHVI"] = home_values.iloc[:, 55:67].mean(axis=1)
home_values["2013-Avg_ZHVI"] = home_values.iloc[:, 67:79].mean(axis=1)
home_values["2014-Avg_ZHVI"] = home_values.iloc[:, 79:91].mean(axis=1)
home_values["2015-Avg_ZHVI"] = home_values.iloc[:, 91:103].mean(axis=1)
home_values["2016-Avg_ZHVI"] = home_values.iloc[:, 103:115].mean(axis=1)
home_values["2017-Avg_ZHVI"] = home_values.iloc[:, 115:127].mean(axis=1)
home_values["2018-Avg_ZHVI"] = home_values.iloc[:, 127:139].mean(axis=1)
home_values["2019-Avg_ZHVI"] = home_values.iloc[:, 139:150].mean(axis=1)

home_sales["2008-Avg_MSR"] = home_sales.iloc[:, 4:14].mean(axis=1)
home_sales["2009-Avg_MSR"] = home_sales.iloc[:, 14:26].mean(axis=1)
home_sales["2010-Avg_MSR"] = home_sales.iloc[:, 26:38].mean(axis=1)
home_sales["2011-Avg_MSR"] = home_sales.iloc[:, 38:50].mean(axis=1)
home_sales["2012-Avg_MSR"] = home_sales.iloc[:, 50:62].mean(axis=1)
home_sales["2013-Avg_MSR"] = home_sales.iloc[:, 62:74].mean(axis=1)
home_sales["2014-Avg_MSR"] = home_sales.iloc[:, 74:86].mean(axis=1)
home_sales["2015-Avg_MSR"] = home_sales.iloc[:, 86:98].mean(axis=1)
home_sales["2016-Avg_MSR"] = home_sales.iloc[:, 98:110].mean(axis=1)
home_sales["2017-Avg_MSR"] = home_sales.iloc[:, 110:122].mean(axis=1)
home_sales["2018-Avg_MSR"] = home_sales.iloc[:, 122:134].mean(axis=1)
home_sales["2019-Avg_MSR"] = home_sales.iloc[:, 134:145].mean(axis=1)

number_sales["2008-Avg_NS"] = number_sales.iloc[:, 4:14].mean(axis=1)
number_sales["2009-Avg_NS"] = number_sales.iloc[:, 14:26].mean(axis=1)
number_sales["2010-Avg_NS"] = number_sales.iloc[:, 26:38].mean(axis=1)
number_sales["2011-Avg_NS"] = number_sales.iloc[:, 38:50].mean(axis=1)
number_sales["2012-Avg_NS"] = number_sales.iloc[:, 50:62].mean(axis=1)
number_sales["2013-Avg_NS"] = number_sales.iloc[:, 62:74].mean(axis=1)
number_sales["2014-Avg_NS"] = number_sales.iloc[:, 74:86].mean(axis=1)
number_sales["2015-Avg_NS"] = number_sales.iloc[:, 86:98].mean(axis=1)
number_sales["2016-Avg_NS"] = number_sales.iloc[:, 98:110].mean(axis=1)
number_sales["2017-Avg_NS"] = number_sales.iloc[:, 110:122].mean(axis=1)
number_sales["2018-Avg_NS"] = number_sales.iloc[:, 122:134].mean(axis=1)
number_sales["2019-Avg_NS"] = number_sales.iloc[:, 134:145].mean(axis=1)

foreclosure_resales["2008-Avg_FR"] = foreclosure_resales.iloc[:, 4:14].mean(axis=1)
foreclosure_resales["2009-Avg_FR"] = foreclosure_resales.iloc[:, 14:26].mean(axis=1)
foreclosure_resales["2010-Avg_FR"] = foreclosure_resales.iloc[:, 26:38].mean(axis=1)
foreclosure_resales["2011-Avg_FR"] = foreclosure_resales.iloc[:, 38:50].mean(axis=1)
foreclosure_resales["2012-Avg_FR"] = foreclosure_resales.iloc[:, 50:62].mean(axis=1)
foreclosure_resales["2013-Avg_FR"] = foreclosure_resales.iloc[:, 62:74].mean(axis=1)
foreclosure_resales["2014-Avg_FR"] = foreclosure_resales.iloc[:, 74:86].mean(axis=1)
foreclosure_resales["2015-Avg_FR"] = foreclosure_resales.iloc[:, 86:98].mean(axis=1)
foreclosure_resales["2016-Avg_FR"] = foreclosure_resales.iloc[:, 98:110].mean(axis=1)
foreclosure_resales["2017-Avg_FR"] = foreclosure_resales.iloc[:, 110:122].mean(axis=1)
foreclosure_resales["2018-Avg_FR"] = foreclosure_resales.iloc[:, 122:134].mean(axis=1)
foreclosure_resales["2019-Avg_FR"] = foreclosure_resales.iloc[:, 134:145].mean(axis=1)

number_sales.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2010-Avg_NS,2011-Avg_NS,2012-Avg_NS,2013-Avg_NS,2014-Avg_NS,2015-Avg_NS,2016-Avg_NS,2017-Avg_NS,2018-Avg_NS,2019-Avg_NS
0,61639,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.000000,69.500000,62.083333,57.363636
1,84654,60657,Illinois,2,134.0,172.0,143.0,184.0,229.0,162.0,...,109.583333,95.5,111.083333,154.75,149.416667,181.583333,190.333333,213.000000,179.750000,131.090909
2,61637,10023,New York,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,75.600000,75.333333,77.500000,71.909091
3,91982,77494,Texas,4,61.0,77.0,85.0,99.0,122.0,95.0,...,79.833333,98.5,123.416667,165.75,162.416667,161.166667,222.500000,257.666667,253.916667,242.400000
4,84616,60614,Illinois,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215.400000,197.416667,153.000000


In [4]:
# Python Dictionary to translate US States to Two letter codes
# Dictionary obtained from rogerallen (https://gist.github.com/rogerallen/1583593)
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Rename StateName columns
home_values = home_values.rename(columns={"RegionName":"ZIP Code",
                                          "2008-01":"2008-01_ZHVI",
                                          "2008-02":"2008-02_ZHVI"})
home_sales = home_sales.rename(columns={"RegionName":"ZIP Code",
                                        "StateName":"State"})
number_sales = number_sales.rename(columns={"RegionName":"ZIP Code",
                                            "StateName":"State",
                                            "seasAdj":"seasAdj_NS"})
foreclosure_resales = foreclosure_resales.rename(columns={"RegionName":"ZIP Code",
                                                          "StateName":"State"})
unemployment = unemployment.rename(columns={"State Abbreviation":"State"})

# Drop columns
unemployment.drop(columns=["Data", "Series Id", "Adjusted?", "Area",
                           "Area Type", "State/Region/Division", "Measure", "Years"])

# Translate states into two letter codes
try:
    for index, row in home_sales.iterrows():
        state = home_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        home_sales.loc[index, "State"] = abbr

    
    for index, row in number_sales.iterrows():
        state = number_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        number_sales.loc[index, "State"] = abbr

    for index, row in foreclosure_resales.iterrows():
        state = foreclosure_resales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        foreclosure_resales.loc[index, "State"] = abbr
except:
    pass

home_values.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank,2008-01_ZHVI,2008-02_ZHVI,2008-03,...,2010-Avg_ZHVI,2011-Avg_ZHVI,2012-Avg_ZHVI,2013-Avg_ZHVI,2014-Avg_ZHVI,2015-Avg_ZHVI,2016-Avg_ZHVI,2017-Avg_ZHVI,2018-Avg_ZHVI,2019-Avg_ZHVI
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1,NaN,NaN,NaN,...,445775.916667,423604.722222,412033.055556,421352.583333,4.435987e+05,4.528587e+05,4.686318e+05,4.814832e+05,4.886200e+05,4.835098e+05
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2,NaN,NaN,NaN,...,836375.277778,858051.166667,888688.861111,949063.444444,1.068758e+06,1.138190e+06,1.197406e+06,1.201872e+06,1.216848e+06,1.176796e+06
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3,269106.0,268326.0,267699.333333,...,274168.222222,266745.861111,271493.027778,290686.722222,3.156699e+05,3.346299e+05,3.319498e+05,3.294792e+05,3.329190e+05,3.340930e+05
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,NaN,...,565302.194444,534765.361111,523258.083333,537427.972222,5.705761e+05,5.916056e+05,6.132054e+05,6.285303e+05,6.361214e+05,6.297365e+05
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5,128064.0,127776.0,127537.666667,...,119721.722222,112142.083333,110622.833333,123323.277778,1.395240e+05,1.541215e+05,1.636677e+05,1.693848e+05,1.760530e+05,1.830033e+05


In [5]:
#Merge the home_values, home_sales, number_sales and foreclosure_resales
merge1 = pd.merge(home_values, home_sales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_ZHVI", "_MSR"))
merge2 = pd.merge(number_sales, foreclosure_resales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_NS", "_FR"))
main_data = pd.merge(merge1, merge2, on=["ZIP Code","State","RegionID"], how="outer")

In [6]:
main_data.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank_ZHVI,2008-01_ZHVI,2008-02_ZHVI,2008-03_ZHVI,...,2010-Avg_FR,2011-Avg_FR,2012-Avg_FR,2013-Avg_FR,2014-Avg_FR,2015-Avg_FR,2016-Avg_FR,2017-Avg_FR,2018-Avg_FR,2019-Avg_FR
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1.0,NaN,NaN,NaN,...,0.000633,0.000550,0.001392,0.025442,0.016950,0.003683,0.005017,0.004208,0.005367,0.006700
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3.0,269106.0,268326.0,267699.333333,...,0.038658,0.029625,0.029517,0.010633,0.004033,0.002458,0.003492,0.002967,0.001367,0.007025
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002430,0.001183,0.006750
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5.0,128064.0,127776.0,127537.666667,...,0.235167,0.206533,0.189350,0.055933,0.031283,0.013625,0.013442,0.012083,0.003492,0.011450


In [7]:
#Function to get mean of the columns from minimum month and year to maximum month and year with different suffixes
#Used regular expressions
def findavg(data,year,suffix):
    x = []
    for i in data.columns:
        y = (re.findall(f'^{year}-([0-9][0-9])_{suffix}',i))
        [x.append(int(i)) for i in y]
    min_col = data.columns.get_loc(f"{year}-{min(x):02d}_{suffix}")
    max_col = data.columns.get_loc(f"{year}-{max(x):02d}_{suffix}")
    data[f"{year}_Avg_{suffix}"] = data.iloc[:,min_col:max_col+1].mean(axis=1)
    return data[f"{year}_Avg_{suffix}"]

In [8]:
# Grabbed the 'RegionID', 'ZIP Code', 'City', 'State', 'CountyName' columns from main_data
new_main_data = main_data[['RegionID', 'ZIP Code', 'City', 'State', 'CountyName']]

new_main_data.head()

,RegionID,ZIP Code,City,State,CountyName
0,84654,60657,Chicago,IL,Cook County
1,61637,10023,New York,NY,New York County
2,91982,77494,Katy,TX,Harris County
3,84616,60614,Chicago,IL,Cook County
4,91940,77449,Katy,TX,Harris County


In [9]:
#Join all indexes across different years to new_main_data
years = [i for i in range(2008,2020,1)]
suffixes = ["ZHVI","MSR","NS","FR"]

for year in years:
    for suffix in suffixes:
        new_main_data = new_main_data.join(findavg(main_data,year,suffix))

In [10]:
new_main_data.head()
new_main_data.to_csv("Output/main_data.csv", index=False, header=True)

In [11]:
#Group the data by State and get mean acroos different zipcodes
state_data =new_main_data[
                     ['State', '2008_Avg_ZHVI','2009_Avg_ZHVI', '2010_Avg_ZHVI', '2011_Avg_ZHVI',
                      '2012_Avg_ZHVI', '2013_Avg_ZHVI', '2014_Avg_ZHVI', '2015_Avg_ZHVI', '2016_Avg_ZHVI',
                      '2017_Avg_ZHVI', '2018_Avg_ZHVI', '2019_Avg_ZHVI', '2008_Avg_MSR','2009_Avg_MSR', 
                      '2010_Avg_MSR', '2011_Avg_MSR', '2012_Avg_MSR','2013_Avg_MSR', '2014_Avg_MSR', 
                      '2015_Avg_MSR', '2016_Avg_MSR','2017_Avg_MSR', '2018_Avg_MSR', '2019_Avg_MSR', 
                      '2008_Avg_NS','2009_Avg_NS', '2010_Avg_NS', '2011_Avg_NS', '2012_Avg_NS','2013_Avg_NS', 
                      '2014_Avg_NS', '2015_Avg_NS', '2016_Avg_NS','2017_Avg_NS', '2018_Avg_NS', '2019_Avg_NS', 
                      '2008_Avg_FR', '2009_Avg_FR', '2010_Avg_FR', '2011_Avg_FR', '2012_Avg_FR','2013_Avg_FR', 
                      '2014_Avg_FR', '2015_Avg_FR', '2016_Avg_FR','2017_Avg_FR', '2018_Avg_FR', '2019_Avg_FR']
                    ].groupby("State").mean()
state_data.head()

,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,2013_Avg_ZHVI,2014_Avg_ZHVI,2015_Avg_ZHVI,2016_Avg_ZHVI,2017_Avg_ZHVI,...,2010_Avg_FR,2011_Avg_FR,2012_Avg_FR,2013_Avg_FR,2014_Avg_FR,2015_Avg_FR,2016_Avg_FR,2017_Avg_FR,2018_Avg_FR,2019_Avg_FR
State,,,,,,,,,,,,,,,,,,,,,
AK,257757.731674,255277.256173,255265.845679,252652.662037,257154.490616,261483.093559,260587.859272,263911.006225,260905.361420,265698.318409,...,0.017953,0.015444,0.013250,0.015368,0.010808,0.014436,0.007723,0.008110,0.008051,0.015248
AL,142287.102771,137912.212703,132560.954521,127438.048756,124757.877755,124216.831304,123857.456570,126152.310004,126710.281543,129905.843310,...,0.028620,0.024541,0.022358,0.036341,0.038174,0.036396,0.029826,0.024794,0.021197,0.018580
AR,109428.506791,105079.607528,102444.345622,100620.504960,100541.646575,101893.995841,102221.952317,102801.309430,104403.715475,106864.871102,...,0.034028,0.024479,0.011072,0.017768,0.024253,0.012681,0.012304,0.011055,0.011784,0.012021
AZ,257791.880246,222270.168886,208525.831265,192496.057547,192090.256004,210370.328371,219667.363194,224545.767161,235508.485779,245929.407627,...,0.138657,0.132389,0.072265,0.059775,0.046469,0.031105,0.021606,0.016913,0.011268,0.009412
CA,468233.711450,423004.034608,422985.651603,401367.360069,406446.894985,446805.162324,476875.116913,503387.102992,536483.483562,569431.051039,...,0.212939,0.205428,0.145055,0.065362,0.036880,0.028936,0.022030,0.013541,0.009642,0.006600


In [22]:
state_data["%Increase/Decrease_ZHVI"] = ((state_data["2019_Avg_ZHVI"] - state_data["2008_Avg_ZHVI"])/state_data["2008_Avg_ZHVI"]) * 100
state_data["%Increase/Decrease_MSR"] = ((state_data["2019_Avg_MSR"] - state_data["2008_Avg_MSR"])/state_data["2008_Avg_MSR"]) * 100
state_data["%Increase/Decrease_NS"] = ((state_data["2019_Avg_NS"] - state_data["2008_Avg_NS"])/state_data["2008_Avg_NS"]) * 100
state_data["%Increase/Decrease_FR"] = ((state_data["2019_Avg_FR"] - state_data["2008_Avg_FR"])/state_data["2008_Avg_FR"]) * 100

In [17]:
def get(base_url, params):
    try:
        return requests.get(base_url, params)
    except Exception:
        # sleep for a bit in case that helps
        time.sleep(1)
        # try again
        return get(base_url, params)

In [23]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#zip_codes = new_main_data["ZIP Code"]

#coordinates = []
params = {
    "key": gkey
}

for index, row in state_data.iterrows():
    try:
        state = index
        params["address"] = state
        response = requests.get(base_url, params)
        json_response = response.json()
        state_data.loc[index, "Lat"] = json_response['results'][0]['geometry']['location']['lat']
        state_data.loc[index, "Lng"] = json_response['results'][0]['geometry']['location']['lng']
        print(state_data.loc[index, ["Lat", "Lng"]])
    except:
        pass
    #except Exception:
        #time.sleep(1)
        #return get(base_url, params).json()

Lat    36.751425
Lng   -95.963661
Name: AK, dtype: float64
Lat    32.318231
Lng   -86.902298
Name: AL, dtype: float64
Lat    36.148173
Lng   -95.972508
Name: AR, dtype: float64
Lat     34.048928
Lng   -111.093731
Name: AZ, dtype: float64
Lat     36.778261
Lng   -119.417932
Name: CA, dtype: float64
Lat    37.036641
Lng   -95.671412
Name: CO, dtype: float64
Lat    41.603221
Lng   -73.087749
Name: CT, dtype: float64
Lat    38.907192
Lng   -77.036871
Name: DC, dtype: float64
Lat    38.910832
Lng   -75.527670
Name: DE, dtype: float64
Lat    26.122439
Lng   -80.137317
Name: FL, dtype: float64
Lat    32.165622
Lng   -82.900075
Name: GA, dtype: float64
Lat    36.148109
Lng   -95.972990
Name: HI, dtype: float64
Lat    41.878003
Lng   -93.097702
Name: IA, dtype: float64
Lat     44.068202
Lng   -114.742041
Name: ID, dtype: float64
Lat    40.633125
Lng   -89.398528
Name: IL, dtype: float64
Lat    40.267194
Lng   -86.134902
Name: IN, dtype: float64
Lat    39.011902
Lng   -98.484246
Name: KS, dtype:

In [24]:
state_data.to_csv("Output/state_data.csv", index=True, header=True)

In [48]:
coordinates = state_data[["Lat", "Lng"]]
pct_increase_msr = state_data['2019_Avg_MSR']


In [53]:
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(coordinates, weights=pct_increase_msr, max_intensity = max(state_data['2019_Avg_MSR']))

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))